In [ ]:
from nbdev import *
%nbdev_default_export utils

Cells will be exported to srthesis.utils,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Utilities



In [ ]:
%nbdev_export_internal
import os
import numpy as np
import torch
from fastai.utils.mem import *

# GPU tools

In [ ]:
%nbdev_export
def get_freer_gpu():
    os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp')
    memory_available = [int(x.split()[2]) for x in open('tmp', 'r').readlines()]
    os.remove("tmp")
    return np.argmax(memory_available)

In [ ]:
%nbdev_export
def select_gpu():
    freer_gpu = get_freer_gpu()
    torch.cuda.set_device(int(freer_gpu))
    torch.cuda.empty_cache()

    print("GPUs:"+str(torch.cuda.device_count()))
    print("Selected GPU:"+str(freer_gpu + 1))
    print("Available memory:" + str(gpu_mem_get_free_no_cache()) + "MB")

In [ ]:
select_gpu()

GPUs:2
Selected GPU:2
Available memory:11005MB


## Debugging

In [ ]:
def pretty_size(size):
	"""Pretty prints a torch.Size object"""
	assert(isinstance(size, torch.Size))
	return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
	"""Prints a list of the Tensors being tracked by the garbage collector."""
	import gc
	total_size = 0
	for obj in gc.get_objects():
		try:
			if torch.is_tensor(obj):
				if not gpu_only or obj.is_cuda:
					print("%s:%s%s %s" % (type(obj).__name__, 
										  " GPU" if obj.is_cuda else "",
										  " pinned" if obj.is_pinned else "",
										  pretty_size(obj.size())))
					total_size += obj.numel()
			elif hasattr(obj, "data") and torch.is_tensor(obj.data):
				if not gpu_only or obj.is_cuda:
					print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
												   type(obj.data).__name__, 
												   " GPU" if obj.is_cuda else "",
												   " pinned" if obj.data.is_pinned else "",
												   " grad" if obj.requires_grad else "", 
												   " volatile" if obj.volatile else "",
												   pretty_size(obj.data.size())))
					total_size += obj.data.numel()
		except Exception as e:
			pass        
	print("Total size:", total_size)

In [ ]:
%nbdev_hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_utils.ipynb.
Converted 0__template.ipynb.
Converted 10_data.ipynb.
Converted 20_metrics.ipynb.
Converted augmentations.ipynb.
Converted sr reference.ipynb.
